In [2]:
import pprint
import numpy as np
import collections
import matplotlib.pyplot as plt

In [10]:
t_slot = 500 / 1000

n_transmissions = 20 # This value represents the number of simultaneous transmissions, similar to number of nodes
alpha = 0.3
# num_slots = 11

for num_slots in [7, 11, 21, 31, 41]:
    t_max = num_slots * t_slot
    mean = t_max * alpha

    slots_total = []
    num_transmissions_per_slot = [0 for _ in range(num_slots)]
    n_iters = 100_000
    for _ in range(n_iters):
        samples = np.random.exponential(mean, n_transmissions) # Select samples from an exponential distribution
        samples = np.clip(samples, 0, t_max) # Since samples can be out of bounds i.e. greater than the last slot we need to clamp it
        samples = t_max - samples # We want a reverse-exponential distribution so that successful transmissions are first
        samples = t_slot * np.floor(samples / t_slot) # Clamp samples to the nearest bottom slot
        
        slots_picked = samples / t_slot # Converts sample which is at a slot time into the actual slot number
        
        slots = [0 for _ in range(num_slots)]
        for slot_idx in slots_picked:
            slots[int(slot_idx)] += 1
            num_transmissions_per_slot[int(slot_idx)] += 1
            
        slots_total.append(slots)
        
    # pprint.pprint(slots_total)

    # With a specific alpha, num_slots, and number of simultaneous transmissions, this is the probability of a collision happening for each slot
    # Compute probability of collision in each slot
    p_collision_per_slot = [0 for _ in range(num_slots)]
    for i in range(num_slots):
        num_slots_used = 0
        num_slots_success = 0
        for j in range(len(slots_total)):
            if slots_total[j][i] > 0:
                num_slots_used += 1
            if slots_total[j][i] == 1:
                num_slots_success += 1
        
        p_collision = (1 - num_slots_success / num_slots_used) if num_slots_used > 0 else -1
        p_collision_per_slot[i] = p_collision

    # print(f"p_collision_per_slot={[f'{p_collision:.3f}' for p_collision in p_collision_per_slot]}")
    # print(f"p_collision_per_slot={[f'{(p_collision * 100):.1f}%' for p_collision in p_collision_per_slot]}")

    # Compute total number of transmissions per slot
    # print(f"num_transmissions_per_slot={num_transmissions_per_slot}")

    # Compute number of successful transmissions
    num_successful_transmissions = 0
    for slot_idx in range(num_slots):
        num_successful_transmissions += int(num_transmissions_per_slot[slot_idx] * (1 - p_collision_per_slot[slot_idx]))

    # print(f"num_successful_transmissions={num_successful_transmissions}")

    num_successful_transmissions_per_round = num_successful_transmissions / n_iters
    # print(f"num_successful_transmissions_per_round={num_successful_transmissions_per_round}")

    total_num_transmissions = n_transmissions * n_iters
    num_collisions = total_num_transmissions - num_successful_transmissions
    # print(f"num_collisions={num_collisions}")
    p_collision = num_collisions / total_num_transmissions
    print(f"p_collision={p_collision}")

    num_successful_transmissions_per_slot = num_successful_transmissions / n_iters / num_slots
    print(f"num_successful_transmissions_per_slot={num_successful_transmissions_per_slot}")



    # ### With elastic slot sizing
    # print()
    # elastic_factor = 2
    # 
    # # Compute number of successful transmissions, similar to throughput, removing successful transmissions in elastic slots
    # num_successful_transmissions_elastic = 0
    # for slot_idx in range(num_slots - elastic_factor):
    #     num_successful_transmissions_elastic += int(num_transmissions_per_slot[slot_idx] * (1 - p_collision_per_slot[slot_idx]))
    # 
    # print(f"num_successful_transmissions={num_successful_transmissions_elastic}")
    # 
    # num_successful_transmissions_per_round_elastic = num_successful_transmissions_elastic / n_iters
    # print(f"num_successful_transmissions_per_round={num_successful_transmissions_per_round_elastic}")
    # 
    # # Compute number of adjusted collisions after remove elastic slots
    # total_num_transmissions_elastic = sum(num_transmissions_per_slot[:len(num_transmissions_per_slot) - elastic_factor])
    # num_collisions_elastic = total_num_transmissions_elastic - num_successful_transmissions_elastic
    # print(f"num_collisions={num_collisions_elastic}")
    # 
    # # Compute adjusted probability of collision assuming elastic slots always end in collision
    # p_collision_elastic = ((n_transmissions * n_iters) - num_successful_transmissions_elastic) / (n_transmissions * n_iters)
    # print(f"p_collision={p_collision_elastic}")
    # 
    # 
    # ### Comparison metrics
    # print()
    # 
    # num_transmissions_saved = num_collisions - num_collisions_elastic
    # print(f"num_transmissions_saved={num_transmissions_saved}")
    # percent_transmissions_saved = (num_collisions - num_collisions_elastic) / num_collisions * 100
    # print(f"percent_transmissions_saved={percent_transmissions_saved:.2f}%")
    # 
    # num_successful_transmissions_lost = num_successful_transmissions - num_successful_transmissions_elastic
    # print(f"num_successful_transmissions_lost={num_successful_transmissions_lost}")
    # 
    # print("\n\n")

p_collision=0.8528705
num_successful_transmissions_per_slot=0.42037
p_collision=0.754388
num_successful_transmissions_per_slot=0.4465672727272727
p_collision=0.552675
num_successful_transmissions_per_slot=0.4260238095238095
p_collision=0.424365
num_successful_transmissions_per_slot=0.37137741935483876
p_collision=0.3427155
num_successful_transmissions_per_slot=0.32062658536585364
